https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

In [1]:
%reload_kedro

2023-10-08 11:24:02,908 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\Dushku\PycharmProjects\monash-datahack-2023
2023-10-08 11:24:03,027 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-10-08 11:24:03,047 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-telemetry-0.2.5
2023-10-08 11:24:03,105 - kedro.extras.extensions.ipython - INFO - ** Kedro project Monash datahack 2023
2023-10-08 11:24:03,105 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\kedro\framework\context\context.py:344: UserWarning: Credentials not found in your Kedro project config.
No files found in ['C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\base', 'C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [2]:
df = catalog.load('encoded_data_with_total_classification')

2023-10-08 11:24:03,109 - kedro.io.data_catalog - INFO - Loading data from `encoded_data_with_total_classification` (CSVDataSet)...


In [3]:
df.head()

,y,Jurisdiction_encoded,Sub-stream,Total type,Category_encoded,Type_encoded,Management_encoded,Fate_encoded,ds_encoded
0,1139,0,3,Biosolids total,1,1,3,0,14
1,0,0,3,Biosolids total,1,1,4,3,14
2,1139,0,9,Biosolids total,1,1,3,0,14
3,0,0,9,Biosolids total,1,1,4,3,14
4,0,0,2,Hazwaste total,2,1,1,1,14


In [4]:
df.columns

Index(['y', 'Jurisdiction_encoded', 'Sub-stream', 'Total type',
       'Category_encoded', 'Type_encoded', 'Management_encoded',
       'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4901 entries, 0 to 4900
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   y                     4901 non-null   int64 
 1   Jurisdiction_encoded  4901 non-null   int64 
 2   Sub-stream            4901 non-null   int64 
 3   Total type            4901 non-null   object
 4   Category_encoded      4901 non-null   int64 
 5   Type_encoded          4901 non-null   int64 
 6   Management_encoded    4901 non-null   int64 
 7   Fate_encoded          4901 non-null   int64 
 8   ds_encoded            4901 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 344.7+ KB


In [6]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split

In [7]:
X = df.loc[:, df.columns != 'y']
y = df['y']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13
)

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

In [9]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


ValueError: could not convert string to float: 'Hazwaste total'

In [ ]:
reg.score(X_test, y_test)

In [ ]:
y_pred = reg.predict(X_test)

predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = mean_squared_error(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params["n_estimators"]) + 1,
    reg.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance")
fig.tight_layout()
plt.show()